In [34]:
#clean both csv's and prepare them for use in the model
import pandas as pd

In [35]:
original_df = pd.read_csv('Resources/heart_2022_no_nans.csv')
original_df.head()

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,Alabama,Female,Very good,4.0,0.0,Within past year (anytime less than 12 months ...,Yes,9.0,None of them,No,...,1.60,71.67,27.99,No,No,Yes,Yes,"Yes, received Tdap",No,No
1,Alabama,Male,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,6.0,None of them,No,...,1.78,95.25,30.13,No,No,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,No
2,Alabama,Male,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,No,8.0,"6 or more, but not all",No,...,1.85,108.86,31.66,Yes,No,No,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
3,Alabama,Female,Fair,5.0,0.0,Within past year (anytime less than 12 months ...,Yes,9.0,None of them,No,...,1.70,90.72,31.32,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
4,Alabama,Female,Good,3.0,15.0,Within past year (anytime less than 12 months ...,Yes,5.0,1 to 5,No,...,1.55,79.38,33.07,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,No


In [36]:
#make a df out of only cols that contain binary values
clean_binary_df = original_df.copy()

#rename cols to be similar to compare easily w/ 2020 csv
clean_binary_df.rename(columns={'SleepHours': 'HoursOfSleep'}, inplace=True)

#drop any cols that have more than 2 categorical vars 
columns_to_drop = ["State", "GeneralHealth", "PhysicalHealthDays", "MentalHealthDays", "LastCheckupTime", "HoursOfSleep", 
                   "RemovedTeeth", "HadDiabetes", "SmokerStatus", "ECigaretteUsage", "RaceEthnicityCategory", "AgeCategory", 
                   "HeightInMeters", "WeightInKilograms", "BMI", "TetanusLast10Tdap"]
                
clean_binary_df.drop(columns=columns_to_drop, inplace=True)

In [37]:
#use one-hot encoding to make categorical columns numerical for binary classification
clean_binary_df = pd.get_dummies(clean_binary_df)
clean_binary_df.head()

,Sex_Female,Sex_Male,PhysicalActivities_No,PhysicalActivities_Yes,HadHeartAttack_No,HadHeartAttack_Yes,HadAngina_No,HadAngina_Yes,HadStroke_No,HadStroke_Yes,...,HIVTesting_Yes,FluVaxLast12_No,FluVaxLast12_Yes,PneumoVaxEver_No,PneumoVaxEver_Yes,HighRiskLastYear_No,HighRiskLastYear_Yes,CovidPos_No,CovidPos_Tested positive using home test without a health professional,CovidPos_Yes
0,True,False,False,True,True,False,True,False,True,False,...,False,False,True,False,True,True,False,True,False,False
1,False,True,False,True,True,False,True,False,True,False,...,False,False,True,False,True,True,False,True,False,False
2,False,True,True,False,True,False,True,False,True,False,...,False,True,False,False,True,True,False,False,False,True
3,True,False,False,True,True,False,True,False,True,False,...,False,False,True,False,True,True,False,False,False,True
4,True,False,False,True,True,False,True,False,True,False,...,False,False,True,False,True,True,False,True,False,False


In [40]:
all_cols_2022_df = original_df.copy()
#remove cols & "State" col (can't add 50 new T/F cols for every state, increases dimensionality way too much)
dropped_cols = ["State", "PhysicalHealthDays", "MentalHealthDays", "HadDiabetes", "TetanusLast10Tdap"]
all_cols_2022_df.drop(columns=dropped_cols, inplace=True)

#manually map cols that have an order that needs to be preserved
all_cols_2022_df['GeneralHealth'] = all_cols_2022_df['GeneralHealth'].map({'Excellent': 4, 'Very Good': 3, 'Good': 2, 'Fair': 1, 'Poor': 0})
all_cols_2022_df['LastCheckupTime'] = all_cols_2022_df['LastCheckupTime'].map({'Within past year (anytime less than 12 months ago)': 0, 
'Within past 2 years (1 year but less than 2 years ago)': 1, 'Within past 5 years (2 years but less than 5 years ago)': 2, '5 or more years ago': 3})
all_cols_2022_df['RemovedTeeth'] = all_cols_2022_df['RemovedTeeth'].map({'None of them': 0, '1 to 5': 1, '6 or more, but not all': 2, 'All': 3})
all_cols_2022_df['SmokerStatus'] = all_cols_2022_df['SmokerStatus'].map({'Never smoked': 0, 'Former smoker': 1, 'Current smoker - now smokes some days': 2, 'Current smoker - now smokes every day': 3})
all_cols_2022_df['ECigaretteUsage'] = all_cols_2022_df['ECigaretteUsage'].map({'Never used e-cigarettes in my entire life': 0, 'Not at all (right now)': 1, 'Use them some days': 2, 'Use them every day': 3})
all_cols_2022_df['AgeCategory'] = all_cols_2022_df['AgeCategory'].map({'Age 18 to 24': 0, 'Age 25 to 29': 1, 'Age 30 to 34': 2, 'Age 35 to 39': 3, 'Age 40 to 44': 4, 'Age 45 to 49': 5, 'Age 50 to 54': 6,
'Age 55 to 59': 7, 'Age 60 to 64': 8, 'Age 65 to 69': 9, 'Age 70 to 74': 10, 'Age 75 to 79': 11, 'Age 80 or older': 12})

not_onehot_encoded = ["GeneralHealth", "LastCheckupTime", "RemovedTeeth", "SmokerStatus", "ECigaretteUsage", "AgeCategory"]

#onehot encode remaining cols
remaining_columns = [col for col in all_cols_2022_df.columns if col not in not_onehot_encoded]
one_hot_encoded_remaining = pd.get_dummies(all_cols_2022_df[remaining_columns])
combined_df = pd.concat([all_cols_2022_df[not_onehot_encoded], one_hot_encoded_remaining], axis=1)

#remove NaN values just in case
combined_df.dropna(inplace=True)
combined_df.head()

,GeneralHealth,LastCheckupTime,RemovedTeeth,SmokerStatus,ECigaretteUsage,AgeCategory,SleepHours,HeightInMeters,WeightInKilograms,BMI,...,HIVTesting_Yes,FluVaxLast12_No,FluVaxLast12_Yes,PneumoVaxEver_No,PneumoVaxEver_Yes,HighRiskLastYear_No,HighRiskLastYear_Yes,CovidPos_No,CovidPos_Tested positive using home test without a health professional,CovidPos_Yes
3,1.0,0,0,0,0,12,9.0,1.70,90.72,31.32,...,False,False,True,False,True,True,False,False,False,True
4,2.0,0,1,0,0,12,5.0,1.55,79.38,33.07,...,False,False,True,False,True,True,False,True,False,False
5,2.0,0,0,0,0,6,7.0,1.85,120.20,34.96,...,True,False,True,True,False,True,False,True,False,False
6,2.0,0,2,0,0,12,8.0,1.63,88.00,33.30,...,False,False,True,False,True,True,False,True,False,False
7,1.0,0,1,0,0,11,8.0,1.75,74.84,24.37,...,True,False,True,False,True,True,False,False,False,True
